In [ ]:
import tensorflow as tf
from tensorflow.keras import models
import numpy as np 
import pandas as pd
import cv2
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [ ]:
letters_model = models.load_model("/content/Letters_model.h5")
emogi_model = models.load_model("/content/Emogis_model (1).h5")

In [ ]:
emogi_map = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 7}

letters_map = {0: 'H', 1: 'I', 2: 'J', 3: 'K', 4: 'N', 5: 'T', 6: 'W', 7: 'X', 8: 'Y', 9: 'Z', 10: 'f', 11: 'r', 12: 't'}

In [ ]:
input_dir = "/content/input_dir"
output_dir = "/content/output_dir"

In [ ]:
def make_border(img):
    white = [255,255,255]
    _, img = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    constant= cv2.copyMakeBorder(img,30,30,30,30,cv2.BORDER_CONSTANT,value=white)
    return constant

In [ ]:
def sharpen_image(img):
    kernel = np.ones((3, 3),np.uint8)
    erosion = cv2.morphologyEx(img, cv2.MORPH_ERODE, kernel, iterations = 2)
    #cv2.imshow('erosion', erosion )

    _, thresh1 = cv2.threshold(erosion, 127, 255, cv2.THRESH_BINARY)
    blur = cv2.blur(thresh1, (2, 2))
    #cv2.imshow('blur', blur)

    _, thresh2 = cv2.threshold(blur, 145, 255, cv2.THRESH_BINARY)
    #cv2.imshow('Thresh', thresh2)

    #kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
    final = cv2.filter2D(thresh2, -1, kernel)
    return final

In [ ]:
def shorten_image(img):
    #img = cv2.imread(img, cv2.IMREAD_GRAYSCALE)
    height, width = img.shape
    _, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
    horizontal_projection = np.sum(thresh, axis=1)
    horizontal_projection = horizontal_projection/255
    blankImage = np.zeros_like(img)
    for i, value in enumerate(horizontal_projection):
        cv2.line(blankImage, (0, i), (width-int(value), i), (255, 255, 255), 1)

    size = horizontal_projection.size
    a = []
    for i in range(size-1):
        if horizontal_projection[i] == 0 and horizontal_projection[i+1] != 0:
            a.append(i)
        elif horizontal_projection[i] != 0 and horizontal_projection[i+1] == 0:
            a.append(i)
    
    if len(a) > 0:
     return thresh[a[0]:a[len(a)-1]]
    return a

In [ ]:
def preprocess_image(input, output_dir):
 img = cv2.imread(input)
 scale_percent = 60 # percent of original size
 width = int(img.shape[1] * scale_percent / 100)
 height = int(img.shape[0] * scale_percent / 100)
 dim = (width, height)
 img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
 height, width, _ = img.shape
 img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 ret, thresh = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
 #cv2.imshow('Thresh', thresh)
 vertical_projection = np.sum(thresh, axis=0)
 vertical_projection = vertical_projection/255
 #print(vertical_projection[0])
 blankImage = np.zeros_like(img)
 for i, value in enumerate(vertical_projection):
     #print(value)
     cv2.line(blankImage, (i, 0), (i, height-int(value)), (255, 255, 255), 1)
 size = vertical_projection.size
 a = []
 for i in range(size-1):
     if vertical_projection[i] == 0 and vertical_projection[i+1] != 0:
         a.append(i)
     elif vertical_projection[i] != 0 and vertical_projection[i+1] == 0:
         a.append(i)
 #print(a)
 #cv2.imshow("New Histogram Projection", blankImage)
 s = 0
 for i in range(0, len(a)-1, 2):
     img_convert = thresh[:, a[i]:a[i+1]]
     img_convert = shorten_image(img_convert)
     if len(img_convert) != 0:
      h, w = img_convert.shape
      x = h*w
      if x>10:
       _, img_convert = cv2.threshold(img_convert, 127, 255, cv2.THRESH_BINARY_INV)
       img_convert = make_border(img_convert)
       img_convert = sharpen_image(img_convert)
       img_convert = cv2.resize(img_convert, (28, 28))
      #  img_convert = imutils.resize(img_convert, height = 28, width = 28)
       #cv2.imwrite(str(s)+".jpg",img_convert)
       cv2.imwrite(os.path.join(output_dir, str(s)+'.jpg'), img_convert)
       #img_convert = cv2.cvtColor(img_convert, cv2.COLOR_BGR2GRAY)
       # _, Thresh = cv2.threshold(img_convert, 127, 255, cv2.THRESH_BINARY_INV)
       # cv2.imshow(str(s), Thresh)
       s += 1

 #cv2.imshow("solon", vert_image[0])
 #cv2.waitKey(0)
 #cv2.destroyAllWindows()

In [ ]:
def char_in_image(file_path):
  img = tf.keras.utils.load_img(
        file_path,
        grayscale=False,
        color_mode='grayscale',
        target_size=(28,28,1),
        interpolation='nearest'
  )
  img_array = tf.keras.utils.img_to_array(img)
  # print(img_array.shape)
  img_array = img_array.reshape(-1,28, 28, 1)
  # img_array = img_array/255.0
  # img_array_1 = img_array.astype('float32')/255.
  # img_array_1 = tf.expand_dims(img_array_1, 0)
  # img_array = tf.expand_dims(img_array, 0)
  letter_predictions = letters_model.predict((img_array.astype('float32'))/255.).flatten(order='C')
  emogi_predictions = emogi_model.predict((img_array.astype('float32'))/255.).flatten(order='C')
  # print(emogi_map[emogi_predictions.argmax()]," " , letters_map[letter_predictions.argmax()])
  # print(letter_predictions)
  # print(emogi_predictions)
  if(max(emogi_predictions)>max(letter_predictions) and max(letter_predictions)<=0.85):
      print(emogi_map[emogi_predictions.argmax(axis=0)])
  else:
      print(letters_map[letter_predictions.argmax(axis=0)])

In [ ]:

for filename in os.listdir(input_dir):
    f = os.path.join(input_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        preprocess_image(input = f, output_dir = output_dir)
# in input, specify path to image
# in output, specify the directory where the images have to be saved in colab workspace

In [ ]:
for filename in os.listdir(output_dir):
    f = os.path.join(output_dir, filename)
    # checking if it is a file
    if os.path.isfile(f):
        img = cv2.imread(f)
        cv2_imshow(img)
        char_in_image(f)

4   W
[1.5496500e-02 3.8409192e-04 3.4261093e-01 9.9922833e-04 4.0672496e-02
 7.0025388e-04 5.0669032e-01 1.1675908e-04 7.8294039e-02 5.3235106e-03
 8.2472824e-03 8.6758548e-05 3.7789161e-04]
[3.2767440e-07 1.6848241e-01 1.0884382e-04 8.2699531e-01 7.4683078e-07
 2.3087594e-03 2.1034929e-03]
4


5   W
[1.09977111e-06 9.64445429e-13 1.71432628e-08 4.98451556e-08
 2.77609928e-07 2.95369257e-10 9.99986529e-01 1.32616085e-10
 1.20201039e-05 1.81502802e-09 1.32264885e-11 1.22880062e-09
 1.55882390e-11]
[0.0000000e+00 1.4837981e-15 1.9533313e-07 5.3607924e-17 9.9679071e-01
 3.2087974e-03 2.6466176e-07]
W


7   W
[1.8981175e-02 4.3849459e-06 2.7213559e-01 4.5433952e-04 1.0389218e-01
 8.1807790e-05 4.8437771e-01 1.0104601e-06 1.1924529e-01 4.7092600e-04
 2.9163304e-04 5.6080386e-05 7.8435733e-06]
[0.0000000e+00 9.6217911e-15 3.4901095e-19 8.9164942e-18 2.0372713e-14
 9.0954484e-17 1.0000000e+00]
7


6   W
[4.0298009e-06 1.1495812e-08 2.7767013e-09 7.0830006e-03 4.9142144e-04
 1.7125564e-09 9.9232519e-01 7.5051059e-09 2.0244896e-08 1.2521942e-10
 1.4011767e-05 8.2392886e-05 6.4331238e-09]
[0.0000000e+00 1.2458499e-30 1.1153005e-21 2.2490339e-23 2.4415643e-04
 9.9975580e-01 7.6114632e-30]
W


3   Y
[3.9009615e-03 1.7755255e-04 4.5810756e-03 1.4834082e-03 7.5778516e-05
 4.7421254e-05 8.3904254e-04 5.4570445e-04 7.0705867e-01 2.6827040e-01
 1.2930049e-02 2.1663380e-05 6.8239540e-05]
[9.8900981e-33 7.6027646e-08 9.3908095e-01 2.0640516e-09 6.5775849e-03
 1.6528319e-06 5.4339688e-02]
3
